# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Applied Data Science Capstone by IBM/Coursera

## Table of contents

 - Introduction: Business Problem

 - Data

 - Methodology

 - Analysis

 - Results and Discussion

 - Conclusion

<br>

## Introduction: Business Problem

In this project we will try to find the best apartment for rent due to a job offer. This project is targeted to everyone that might be interested to know more about rent prices of apartments, statistics of the neighborhoods or just want to move to São Paulo, Brazil.

Since our job offer it is for a specific address, we have selected a few close neighborhoods to live in. Although we have selected these neighborhoods, still there are a lot of apartments for rent. We have a few exigences regarding the nearby venues, and that will be taken into account to choose our home.

We will use our techniques in data science to generate the most promissing apartments for us to live in. Economic data and social characteristics of the neighborhoods will be considered in the proccess.

<br>

## Data

Based on definition of our problem, factors that will influence our decision are:

 - the apartment must have at least 2 bedrooms
 - the monthly value of the rent cannot exceed Dollars 1.700,00
 - it has to be at least 5km from workplace
 - it has to be near at least 1km from a gym and a market
 - social and economic characteristics of the neighborhod based on the available data

We decided to select a few neighborhoods around our workplace to extract our data.

Following data sources will be needed to extract/generate the required information:

 - coordinate of workplace will be obtained using geocoder
 - economical and location data of rental properties around the workplace will be obtained through webscraping of a major online portal.
 - nearby venues around each neighborhood will be obtained using Foursquare API

<br>

## Let's load our libraries and our apartments data

In [25]:
!pip install folium
!pip install geocoder
!pip install haversineimport pandas as pd
import numpy as np
import json
import requests
from haversine import haversine
import geocoder
import requests
import folium
from sklearn.cluster import KMeans

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: 

In [24]:
# Before we start, let's just define out basic factors.
starting_point = 'Rua Girassol, 555, Vila Madalena, São Paulo/SP'
max_rent_value = 1700
min_bedrooms = 2
min_dist_workplace = 5
min_dist_gym = 1
min_dist_mkt = 1

In [ ]:
dataset = pd.read_csv('rent_properties_data.csv')

In [ ]:
dataset.shape

Let's just reassure that all of the properties are apartments.

In [ ]:
dataset['type'].value_counts()

In [ ]:
dataset.drop(columns=['type'], inplace=True)

Now, let's get our workplace coordinates in order to get the nearby venues.

In [ ]:
work_location = geocoder.arcgis(starting_point).latlng

In [ ]:
work_location

In [ ]:
dataset.head()

In [ ]:
dataset = dataset.loc[(dataset['latitude'] != '-') & (dataset['longitude'] != '-')]

In [ ]:
dataset.shape

Now we have to measure the distance between each apartment to our workplace. First, let's create a function to merge latitude and longitude in a single row.

In [ ]:
def get_location(dataframe, name_latitude, name_longitude):
    df = dataframe.copy()
    df['location'] = df[name_latitude].astype(str) + ',' + df[name_longitude].astype(str)
    df['location'] = df['location'].apply(lambda x: x.split(','))
    df['location'] = df['location'].apply(lambda x: [float(i) for i in x])
    return df

In [ ]:
dataset = get_location(dataset, name_latitude='latitude', name_longitude='longitude')

In [ ]:
dataset.head()

In [ ]:
dataset['distance_from_workplace'] = dataset['location'].apply(lambda x: haversine(work_location, x))

In [ ]:
dataset.head()